### Capstone Project

In [1]:
#import libraries

import zipfile
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import scipy.stats
import seaborn as sns
from scipy import stats
import requests

In [2]:
#Download the zip file from URL

url = 'https://ftp.cdc.gov/pub/Health_Statistics/NCHS/Datasets/CHDI/chsi_dataset.zip'
target_path = 'DATA606.zip'

response = requests.get(url, stream=True)
handle = open(target_path, "wb")
for chunk in response.iter_content(chunk_size=512):
    if chunk:  # filter out keep-alive new chunks
        handle.write(chunk)
handle.close()

In [13]:
#define the two dataframes

zf = zipfile.ZipFile('DATA606.zip')
#I have over 140 columns and I will only need select ones which I've defined in the lists below
#demographics
dem_col= ['State_FIPS_Code','County_FIPS_Code', 'CHSI_County_Name','CHSI_State_Name', 'CHSI_State_Abbr', 'Strata_ID_Number', 'Population_Size', 'Age_19_Under', 'Age_19_64', 'Age_65_84', 'Age_85_and_Over']
#PreventiveServicesUse
prev_col= ['State_FIPS_Code','County_FIPS_Code', 'CHSI_County_Name','CHSI_State_Name', 'CHSI_State_Abbr', 'Strata_ID_Number', 'Pap_Smear', 'Mammogram', 'Proctoscopy', 'Pneumo_Vax', 'Flu_Vac']
#MeasuresOfBirthAndDeath
mbd_col= ['State_FIPS_Code','County_FIPS_Code', 'CHSI_County_Name','CHSI_State_Name', 'CHSI_State_Abbr', 'Strata_ID_Number', 'LBW', 'Premature', 'Late_Care', 'Infant_Mortality', 'Brst_Cancer', 'Col_Cancer', 'CHD', 'Lung_Cancer', 'Stroke', ]
#RiskFactorsAndAccessToCare
risk_col= ['State_FIPS_Code', 'County_FIPS_Code', 'CHSI_County_Name', 'CHSI_State_Name', 'CHSI_State_Abbr', 'Strata_ID_Number', 'Obesity', 'High_Blood_Pres', 'Diabetes', 'Elderly_Medicare','Disabled_Medicare', 'Uninsured', 'Prim_Care_Phys_Rate', 'Dentist_Rate', 'Community_Health_Center_Ind']

#extract csv file from zip file/define dataframes
df0= pd.read_csv(zf.open('DEMOGRAPHICS.csv'), usecols= dem_col)
df1 = pd.read_csv(zf.open('PREVENTIVESERVICESUSE.csv'), usecols= prev_col)
df2 = pd.read_csv(zf.open('MEASURESOFBIRTHANDDEATH.csv'), usecols= mbd_col) 
df3 = pd.read_csv(zf.open('RISKFACTORSANDACCESSTOCARE.csv'), usecols= risk_col)
pd.set_option('display.max_columns', 150)

In [14]:
#Merge df0, df1, df2, and df3
df= df0.merge(df1).merge(df2).merge(df3)
#display 5 first rows
df.head()

,State_FIPS_Code,County_FIPS_Code,CHSI_County_Name,CHSI_State_Name,CHSI_State_Abbr,Strata_ID_Number,Population_Size,Age_19_Under,Age_19_64,Age_65_84,Age_85_and_Over,Pap_Smear,Mammogram,Proctoscopy,Pneumo_Vax,Flu_Vac,LBW,Premature,Late_Care,Infant_Mortality,Brst_Cancer,Col_Cancer,CHD,Lung_Cancer,Stroke,Obesity,High_Blood_Pres,Diabetes,Uninsured,Elderly_Medicare,Disabled_Medicare,Prim_Care_Phys_Rate,Dentist_Rate,Community_Health_Center_Ind
0,1,1,Autauga,Alabama,AL,29,48612,26.9,62.3,9.8,0.9,-1111.1,-1111.1,-1111.1,-1111.1,-1111.1,8.1,14.3,12.9,5.9,34.1,15.9,172.4,65.3,73.9,24.5,29.1,14.2,5690,4762,1209,45.3,22.6,1
1,1,3,Baldwin,Alabama,AL,16,162586,23.5,60.3,14.5,1.8,86.1,82.0,44.1,58.7,61.8,8.6,15.3,15.3,7.1,22.5,15.3,156.5,62.1,48.7,23.6,30.5,7.2,19798,22635,3839,67.0,30.8,1
2,1,5,Barbour,Alabama,AL,51,28414,24.3,62.5,11.6,1.6,-1111.1,-1111.1,-1111.1,-1111.1,-1111.1,11.0,16.7,26.0,7.1,30.5,23.4,95.3,70.6,84.5,25.6,-1111.1,6.6,5126,3288,1092,45.8,24.6,1
3,1,7,Bibb,Alabama,AL,42,21516,24.6,63.3,10.9,1.2,-1111.1,-1111.1,-1111.1,-1111.1,-1111.1,8.7,13.7,16.3,9.8,30.1,17.4,184.7,64.6,144.6,-1111.1,-1111.1,13.1,3315,2390,974,41.8,18.6,1
4,1,9,Blount,Alabama,AL,28,55725,24.5,62.1,12.1,1.3,87.2,-1111.1,-1111.1,-1111.1,-1111.1,7.6,12.8,16.7,9.0,30.2,16.8,148.2,71.7,47.3,24.2,-1111.1,8.4,8131,5019,1300,16.2,10.8,2


In [15]:
#I replace the negative values with NaN so I can have accurate statistics 
df_clean = df.replace({ -1111.1: np.nan, -2222.2: np.nan, -2222.0: np.nan})

In [16]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3141 entries, 0 to 3140
Data columns (total 34 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   State_FIPS_Code              3141 non-null   float64
 1   County_FIPS_Code             3141 non-null   float64
 2   CHSI_County_Name             3141 non-null   object 
 3   CHSI_State_Name              3141 non-null   object 
 4   CHSI_State_Abbr              3141 non-null   object 
 5   Strata_ID_Number             3141 non-null   float64
 6   Population_Size              3141 non-null   float64
 7   Age_19_Under                 3141 non-null   float64
 8   Age_19_64                    3141 non-null   float64
 9   Age_65_84                    3141 non-null   float64
 10  Age_85_and_Over              3141 non-null   float64
 11  Pap_Smear                    1548 non-null   float64
 12  Mammogram                    894 non-null    float64
 13  Proctoscopy       

In [17]:
#general statistics 

df_clean.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
State_FIPS_Code,3141.0,30.304680,15.134423,1.0,18.000,29.0,45.000,56.0
County_FIPS_Code,3141.0,103.716651,107.999484,1.0,35.000,79.0,133.000,840.0
Strata_ID_Number,3141.0,44.696275,25.118434,1.0,23.000,44.0,66.000,88.0
Population_Size,3141.0,94368.164279,306431.655763,62.0,11211.000,25235.0,64040.000,9935475.0
Age_19_Under,3141.0,24.806527,3.281777,1.4,22.700,24.6,26.400,47.2
Age_19_64,3141.0,60.289398,3.356056,47.6,58.300,60.3,62.300,83.3
Age_65_84,3141.0,12.789430,3.334035,2.1,10.700,12.5,14.700,29.2
Age_85_and_Over,3141.0,2.115409,0.949119,0.1,1.500,1.9,2.600,7.6
Pap_Smear,1548.0,82.009561,5.353611,58.9,78.600,82.5,85.700,98.2
Mammogram,894.0,80.032550,5.912998,58.3,76.500,80.6,84.100,95.9


In [18]:
df_clean['CHSI_State_Name'].unique()

array(['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California',
       'Colorado', 'Connecticut', 'Delaware', 'District of Columbia',
       'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana',
       'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland',
       'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi',
       'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire',
       'New Jersey', 'New Mexico', 'New York', 'North Carolina',
       'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania',
       'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee',
       'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington',
       'West Virginia', 'Wisconsin', 'Wyoming'], dtype=object)

In [19]:
df_clean.to_csv('updateddata.csv')